# 1D Mirror Detection Learner
**Goal**: correctly identify whether an abstract mirror object is within the vision range

In [1]:
import tensorflow as tf
import numpy

## Define constants

In [16]:
image_width = 2
image_height = 2
vision_range = image_width * image_height
category_count = 10
label_type_count = 2  # mirror or not

segment_count = {
    "training": 8000,
    "test": 1300,
    "validation": 700
    }
sample_count = {
    "training": segment_count['training'] * vision_range,
    "test": segment_count['test'] * vision_range,
    "validation": segment_count['validation'] * vision_range
    }
learning_rate = 0.5

## Auxiliary functions

### Show result

In [3]:
def show_result(x):
    """Print the result"""
    print("{:.0f}%".format(x * 100))

### Automaticaly generate data sets

In [12]:
def mirror_label(x):
    if x is True:
        return [1, 0]
    else:
        return [0, 1]
    
def random_mirror(vision_range, category_count):
    """Return a n-array with either a randomly placed mirror
        or no mirror at all.
    
    Args:
        vision_range (int): how many objects to be seen at a time
        category_count (int): total number of different objects
            including the mirror type
    Return:
        2-tuple:
             [0]: an array of size same as the "vision_range".
             [1]: 1 one-hot 2-array [1, 0] (has mirror) or [0, 1] (no mirror).
    """

    p1 = numpy.random.randint(1, high=category_count, size=vision_range-1)
    p2 = numpy.random.randint(0, category_count, 1)
    data = numpy.random.permutation(numpy.concatenate((p1, p2)))
    return (data, mirror_label(0 in data))



In [6]:
def mirror_data(n, vision_range, category_count):
    """Return a n x size matrix
    Args:
        n (int): number of data points
        vision_range (int): number of objects to be seen at a time
        category_count (int): total number of different object categories
            including the mirror
    Returns:
        A dictionary:
            data: (n, v)-sized 2D numpy array where v is the vision_range
            labels: (n,)-sized 1D numpy array (each element is either 1 or 0)
    """
    raw = [random_mirror(vision_range, category_count) for i in range(n)]
    return {
        "data": numpy.array([x[0] for x in raw]),
        "labels": numpy.array([x[1] for x in raw])
    }

In [18]:
def mirror_data_with_overlap(n, vision_range, category_count):
    """Return a n x size matrix
    Args:
        n (int): number of non-overlapping segments
        vision_range (int): number of objects to be seen at a time
        category_count (int): total number of different object categories
            including the mirror
    Returns:
        A dictionary:
            data: (n, v)-sized 2D numpy array where v is the vision_range
            labels: (n,)-sized 1D numpy array (each element is either 1 or 0)
    """
    
    raw = [random_mirror(vision_range, category_count) for i in range(n)]
    merged = numpy.concatenate([x[0] for x in raw])
    data_set = [merged[i:i+vision_range] for i in range(len(merged)-vision_range)]
    labels = [mirror_label(0 in x) for x in data_set]
    return {
        "data": numpy.array(data_set),
        "labels": labels
    }
    

## Dataset

In [7]:
# dataset = {
#     "training": mirror_data(sample_count['training'], vision_range, category_count),
#     "test": mirror_data(sample_count['test'], vision_range, category_count),
#     "validation": mirror_data(sample_count['validation'], vision_range, category_count)
# }

In [19]:
dataset = {
    "training": mirror_data_with_overlap(segment_count['training'], vision_range, category_count),
    "test": mirror_data_with_overlap(segment_count['test'], vision_range, category_count),
    "validation": mirror_data_with_overlap(segment_count['validation'], vision_range, category_count)
}

In [20]:
dataset['test']['data'][0:15, :]

array([[5, 9, 8, 8],
       [9, 8, 8, 7],
       [8, 8, 7, 9],
       [8, 7, 9, 7],
       [7, 9, 7, 5],
       [9, 7, 5, 1],
       [7, 5, 1, 1],
       [5, 1, 1, 0],
       [1, 1, 0, 8],
       [1, 0, 8, 1],
       [0, 8, 1, 2],
       [8, 1, 2, 7],
       [1, 2, 7, 3],
       [2, 7, 3, 2],
       [7, 3, 2, 7]])

In [21]:
dataset['test']['labels'][0:15]

[[0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1]]

# Memory allocation

In [22]:
x = tf.placeholder(tf.float32, [None, vision_range])

In [33]:
W = tf.Variable(tf.zeros([vision_range, label_type_count]))

In [34]:
b = tf.Variable(tf.zeros([label_type_count]))

In [35]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [36]:
y_ = tf.placeholder(tf.float32, [None, label_type_count])

In [37]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
)

In [38]:
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

In [39]:
# create a session
session = tf.InteractiveSession()

In [40]:
tf.global_variables_initializer().run()

# I. Regular Neural Net

## Training



In [41]:
session.run(train_step, feed_dict={x: dataset['training']['data'], y_: dataset['training']['labels']})

In [42]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

In [43]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Test

In [44]:
test_result = session.run(accuracy, feed_dict={x: dataset['test']['data'], y_: dataset['test']['labels']})

In [45]:
show_result(test_result)

92%


## Validation

In [46]:
exam_result = session.run(accuracy, feed_dict={x: dataset['validation']['data'], y_: dataset['validation']['labels']})

In [47]:
show_result(exam_result)

90%
